## 1. Importação de Bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configuração do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Bibliotecas importadas com sucesso!")

## 2. Carregamento dos Dados

In [ ]:
# Carrega o dataset
data_path = Path('../datasets/mental_health.csv')
df = pd.read_csv(data_path)

print(f"📊 Dataset carregado: {df.shape[0]} linhas × {df.shape[1]} colunas")
df.head()

## 3. Visão Geral dos Dados

In [ ]:
# Informações gerais
print("=" * 60)
print("INFORMAÇÕES DO DATASET")
print("=" * 60)
df.info()
print("\n" + "=" * 60)
print("ESTATÍSTICAS DESCRITIVAS")
print("=" * 60)
df.describe()

In [ ]:
# Análise de valores nulos
null_counts = df.isnull().sum()
null_percentage = (null_counts / len(df)) * 100

null_df = pd.DataFrame({
    'Coluna': null_counts.index,
    'Valores Nulos': null_counts.values,
    'Percentual': null_percentage.values
})
null_df = null_df[null_df['Valores Nulos'] > 0].sort_values('Valores Nulos', ascending=False)

print("\n📋 COLUNAS COM VALORES NULOS:")
print(null_df.to_string(index=False))

## 4. Análise de Idade (Age)

In [ ]:
# Estatísticas de idade
if 'Age' in df.columns:
    print("📊 ESTATÍSTICAS DE IDADE:")
    print(f"  Média: {df['Age'].mean():.1f} anos")
    print(f"  Mediana: {df['Age'].median():.1f} anos")
    print(f"  Desvio Padrão: {df['Age'].std():.1f}")
    print(f"  Mínimo: {df['Age'].min():.0f} anos")
    print(f"  Máximo: {df['Age'].max():.0f} anos")
    
    # Distribuição de idade
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histograma
    axes[0].hist(df['Age'].dropna(), bins=50, edgecolor='black', alpha=0.7)
    axes[0].set_title('Distribuição de Idade', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Idade')
    axes[0].set_ylabel('Frequência')
    axes[0].axvline(df['Age'].mean(), color='red', linestyle='--', label=f'Média: {df["Age"].mean():.1f}')
    axes[0].legend()
    
    # Boxplot
    axes[1].boxplot(df['Age'].dropna(), vert=True)
    axes[1].set_title('Boxplot de Idade', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Idade')
    
    plt.tight_layout()
    plt.show()
    
    # Identificação de outliers
    outliers_count = ((df['Age'] < 18) | (df['Age'] > 100)).sum()
    print(f"\n⚠️ Outliers detectados (idade < 18 ou > 100): {outliers_count} registros")

## 5. Análise de Gênero (Gender)

In [ ]:
# Distribuição de gênero (valores originais)
if 'Gender' in df.columns:
    print("👥 DISTRIBUIÇÃO DE GÊNERO (Top 15 valores mais frequentes):")
    gender_counts = df['Gender'].value_counts().head(15)
    print(gender_counts)
    
    print(f"\n📊 Total de valores únicos: {df['Gender'].nunique()}")
    
    # Visualização
    plt.figure(figsize=(12, 6))
    gender_counts.plot(kind='bar', edgecolor='black', alpha=0.7)
    plt.title('Top 15 Valores de Gênero Mais Frequentes', fontsize=14, fontweight='bold')
    plt.xlabel('Gênero')
    plt.ylabel('Frequência')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

## 6. Análise de Tratamento (Treatment)

In [ ]:
# Análise de busca por tratamento
if 'treatment' in df.columns:
    treatment_counts = df['treatment'].value_counts()
    treatment_pct = (treatment_counts / len(df)) * 100
    
    print("🏥 BUSCA POR TRATAMENTO DE SAÚDE MENTAL:")
    for value, count in treatment_counts.items():
        pct = (count / len(df)) * 100
        print(f"  {value}: {count} ({pct:.1f}%)")
    
    # Gráfico de pizza
    plt.figure(figsize=(8, 8))
    plt.pie(treatment_counts, labels=treatment_counts.index, autopct='%1.1f%%', 
            startangle=90, explode=[0.05, 0])
    plt.title('Distribuição de Busca por Tratamento', fontsize=14, fontweight='bold')
    plt.axis('equal')
    plt.show()

## 7. Análise por País (Country)

In [ ]:
# Top 10 países
if 'Country' in df.columns:
    country_counts = df['Country'].value_counts().head(10)
    
    print("🌍 TOP 10 PAÍSES:")
    print(country_counts)
    
    # Visualização
    plt.figure(figsize=(12, 6))
    country_counts.plot(kind='barh', edgecolor='black', alpha=0.7)
    plt.title('Top 10 Países com Mais Respondentes', fontsize=14, fontweight='bold')
    plt.xlabel('Número de Respondentes')
    plt.ylabel('País')
    plt.tight_layout()
    plt.show()

## 8. Análise de Trabalho Remoto (remote_work)

In [ ]:
# Trabalho remoto vs Tratamento
if 'remote_work' in df.columns and 'treatment' in df.columns:
    remote_treatment = pd.crosstab(df['remote_work'], df['treatment'], normalize='index') * 100
    
    print("💼 TRABALHO REMOTO vs BUSCA POR TRATAMENTO (%)")
    print(remote_treatment)
    
    # Visualização
    remote_treatment.plot(kind='bar', figsize=(10, 6), edgecolor='black', alpha=0.7)
    plt.title('Busca por Tratamento por Modalidade de Trabalho', fontsize=14, fontweight='bold')
    plt.xlabel('Trabalho Remoto')
    plt.ylabel('Percentual (%)')
    plt.xticks(rotation=0)
    plt.legend(title='Tratamento', labels=['Não', 'Sim'])
    plt.tight_layout()
    plt.show()

## 9. Correlação entre Variáveis Numéricas

In [ ]:
# Matriz de correlação para colunas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns

if len(numeric_cols) > 1:
    correlation = df[numeric_cols].corr()
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', 
                square=True, linewidths=0.5, center=0)
    plt.title('Matriz de Correlação', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 10. Insights e Conclusões

In [ ]:
print("="*70)
print("📊 RESUMO DA ANÁLISE EXPLORATÓRIA")
print("="*70)
print(f"\n1. TAMANHO DO DATASET:")
print(f"   - Total de registros: {len(df):,}")
print(f"   - Total de colunas: {df.shape[1]}")

if 'Age' in df.columns:
    print(f"\n2. PERFIL DEMOGRÁFICO:")
    print(f"   - Idade média: {df['Age'].mean():.1f} anos")
    print(f"   - Faixa etária: {df['Age'].min():.0f} - {df['Age'].max():.0f} anos")

if 'treatment' in df.columns:
    treatment_yes = (df['treatment'] == 'Yes').sum()
    treatment_pct = (treatment_yes / len(df)) * 100
    print(f"\n3. SAÚDE MENTAL:")
    print(f"   - Buscaram tratamento: {treatment_pct:.1f}%")

if 'Country' in df.columns:
    print(f"\n4. DISTRIBUIÇÃO GEOGRÁFICA:")
    print(f"   - Países representados: {df['Country'].nunique()}")
    print(f"   - País mais representado: {df['Country'].value_counts().index[0]}")

print(f"\n5. QUALIDADE DOS DADOS:")
missing = df.isnull().sum().sum()
missing_pct = (missing / (len(df) * df.shape[1])) * 100
print(f"   - Valores faltantes: {missing:,} ({missing_pct:.2f}% do total)")

print("\n" + "="*70)
print("✅ Análise exploratória concluída!")
print("="*70)